In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
top_games = pd.read_csv("top_games.csv")

In [3]:
l =list(top_games['ID'])

In [4]:
top_games.head()

,Name,Rank,ID
0,Gloomhaven,1,174430
1,Pandemic Legacy: Season 1,2,161936
2,Through the Ages: A New Story of Civilization,3,182028
3,Terraforming Mars,4,167791
4,Twilight Struggle,5,12333


In [20]:
string = ''
domain_types = [ 'Abstract Strategy Games', 'Customizable Games', 'Thematic Games', 'Family Games',
"Children's Games", "Party Games", "Strategy Games", "Wargames"]
game_stats = pd.DataFrame()
count = 0.0
for x in range(100,1001,100):
    for i in l[:x]:
        string += str(i) + ","
    string = string[:-1]
    response_stats = requests.get("https://boardgamegeek.com/xmlapi/boardgame/"+string+"?&stats=1")
    content_stats = response_stats.content
    parser_stats = BeautifulSoup(content_stats, 'html.parser')
    boardgames = parser_stats.findAll("boardgame")
    print(len(boardgames))
    for boardgame in boardgames:
        count+=1
        print(count)    
        temp = {}
#         print(boardgame)
#         print(id)
        print(boardgame['objectid'])
        temp['ID'] = boardgame['objectid']
        temp['Year Published'] = boardgame.yearpublished.text
        temp['Min Players'] = boardgame.minplayers.text
        temp['Max Players'] = boardgame.maxplayers.text
        temp['Playing Time'] = boardgame.playingtime.text
        temp['Min Playing Time'] = boardgame.minplaytime.text
        temp['Max Playing Time'] = boardgame.maxplaytime.text
        temp['Owned'] = boardgame.owned.text
        temp['Users Rated'] = boardgame.usersrated.text
        temp['Average Rating'] = boardgame.average.text
        temp['Bayes Average Rating'] = boardgame.bayesaverage.text
        temp['Age'] = int(boardgame.age.text)
        temp['Complexity Rating'] = float(boardgame.averageweight.text)
        temp['Comments'] = boardgame.numcomments.text
        temp['Wishing'] = boardgame.wishing.text
        temp['NumWeights'] = boardgame.numweights.text
        temp['Wanting'] = boardgame.wanting.text
        temp['Trading'] = boardgame.trading.text

        if (boardgame.videogamebg != None):
            temp['Video Games'] = boardgame.videogamebg.text
        else:
            temp['Video Games'] = ' '

        temp['Median'] = boardgame.median.text
        temp['Std Deviation'] = boardgame.stddev.text

        temp['Publisher'] = boardgame.findAll("boardgamepublisher")[0].text

        publishers = boardgame.findAll("boardgamepublisher")
        game_publishers = []
        for publisher in publishers[1:]:
            game_publishers.append(publisher.text)
        temp['Secondary Publishers'] = game_publishers

        polls = boardgame.findAll("poll")

        poll_numplayers = boardgame.find("poll",{"name":"suggested_numplayers"})

        players_df = pd.DataFrame()
        num = []
        labels = []
        percent = []
        for i in range(1,len(poll_numplayers.findAll("results"))):
            poll_numplayers_1 = poll_numplayers.find("results",{"numplayers":i})
            numvotes_players_1 = []
            values_players_1 = []
            total = 0.0
            for result in poll_numplayers_1.findAll("result"):
                total += int(result['numvotes'])
                numvotes_players_1.append(int(result['numvotes']))
                values_players_1.append(result['value'])
            total = (1.0 if total == 0.0 else total)
            if numvotes_players_1 != []: 
                max_players_1 = max(numvotes_players_1)
                percent_players_1 = (max(numvotes_players_1)/total)*100
                index_players_1 = numvotes_players_1.index(max_players_1)
                num.append(i)
                labels.append(values_players_1[index_players_1])
                percent.append(percent_players_1)
        players_df['num'] = num
        players_df['labels'] = labels
        players_df['percent'] = percent

        for label in players_df['labels'].unique():
            temp['Num Players '+label] = (players_df[players_df['labels'] == label]['num']).tolist()

        poll_lang = boardgame.find("poll",{"name":"language_dependence"})
        numvotes_lang = []
        values_lang = []
        for result in poll_lang.findAll("result"):
            numvotes_lang.append(int(result['numvotes']))
            values_lang.append(result['value'])
        if numvotes_lang != []:
            max_lang = max(numvotes_lang)
            index_lang = numvotes_lang.index(max_lang)
            temp['Language Dependence'] = values_lang[index_lang]
        else:
            temp['Language Dependence'] = ' '

        poll_age = boardgame.find("poll",{"name":"suggested_playerage"})
        numvotes_age = []
        values_age = []
        for result in poll_age.findAll("result"):
            numvotes_age.append(int(result['numvotes']))
            values_age.append(result['value'])
        if numvotes_age !=[]:
            max_age = max(numvotes_age)
            index_age = numvotes_age.index(max_age)
            temp['Suggested Player Age'] = values_age[index_age]
        else:
            temp['Suggested Player Age'] = 0

        domains = []
        boardgamesubdomain = boardgame.findAll("boardgamesubdomain")
        for domain in boardgamesubdomain:
            domains.append(domain.text)
        temp['Game Domains'] = domains

        for domain in domain_types:
            if domain in domains:
                temp[domain] = True
            else:
                temp[domain] = False

        categories = []
        gamecategory = boardgame.findAll("boardgamecategory")
        for category in gamecategory:
            categories.append(category.text)
        temp['Game Categories'] = categories

        mechanics = boardgame.findAll("boardgamemechanic")
        game_mechanics = []
        for mechanic in mechanics:
            game_mechanics.append(mechanic.text)
        temp['Game Mechanics'] = game_mechanics

        designers = boardgame.findAll("boardgamedesigner")
        game_designers = []
        for designer in designers:
            game_designers.append(designer.text)
        temp['Game Designers'] = game_designers

        families = boardgame.findAll("boardgamefamily")
        game_families = []
        for family in families:
            game_families.append(family.text)
        temp['Game Family'] = game_families

        ranks = boardgame.findAll("rank")
    # Should I keep this column in the df or not?
    #     rank_dict={}
    #     for rank in ranks:
    #         rank_dict[rank['friendlyname']]=int(rank['value'])
    #     temp['Ranks'] = rank_dict

        if len(ranks)>1:
            for rank in ranks[1:]:
                temp[rank['friendlyname']]=int(rank['value'])

        artists = boardgame.findAll("boardgameartist")
        game_artists = []
        for artist in artists:
            game_artists.append(artist.text)
        temp['Game Artists'] = game_artists

        temp_df = pd.DataFrame.from_dict(temp, orient='index')
        temp_df = temp_df.transpose()

        game_stats = pd.concat([game_stats, temp_df],sort=False)

100
1.0
174430
2.0
161936
3.0
182028
4.0
167791
5.0
12333
6.0
187645
7.0
169786
8.0
220308
9.0
193738
10.0
120677
11.0
84876
12.0
115746
13.0
173346
14.0
180263
15.0
233078
16.0
3076
17.0
102794
18.0
205637
19.0
183394
20.0
124361
21.0
31260
22.0
96848
23.0
162886
24.0
205059
25.0
28720
26.0
164928
27.0
170216
28.0
175914
29.0
209010
30.0
177736
31.0
164153
32.0
25613
33.0
2651
34.0
55690
35.0
221107
36.0
72125
37.0
230802
38.0
126163
39.0
35677
40.0
121921
41.0
171623
42.0
124742
43.0
224517
44.0
68448
45.0
122515
46.0
216132
47.0
18602
48.0
28143
49.0
62219
50.0
12493
51.0
159675
52.0
178900
53.0
146021
54.0
110327
55.0
157354
56.0
205896
57.0
93
58.0
146508
59.0
163412
60.0
40834
61.0
201808
62.0
132531
63.0
185343
64.0
125153
65.0
172386
66.0
37111
67.0
2511
68.0
73439
69.0
103885
70.0
144733
71.0
42
72.0
150376
73.0
102680
74.0
36218
75.0
30549
76.0
127023
77.0
521
78.0
198928
79.0
104162
80.0
34635
81.0
194655
82.0
175155
83.0
161970
84.0
9609
85.0
155426
86.0
146652
87.0
147020


In [21]:
%%time
game_stats.reset_index(drop=True, inplace=True)

CPU times: user 70 µs, sys: 1 µs, total: 71 µs
Wall time: 74.9 µs


In [22]:
game_stats.to_csv("game_stats_bulk.csv", index=False)

In [23]:
game_stats

,ID,Year Published,Min Players,Max Players,Playing Time,Min Playing Time,Max Playing Time,Owned,Users Rated,Average Rating,...,Game Family,Thematic Rank,Strategy Game Rank,Game Artists,Num Players Not Recommended,War Game Rank,Customizable Rank,Abstract Game Rank,Family Game Rank,Party Game Rank
0,174430,2017,1,4,120,60,120,33576,20312,8.92994,...,"[Campaign Games, Components: Miniatures, Crowd...",1,1,"[Alexandr Elichev, Josh T. McDowell, Alvaro Ne...",NaN,NaN,NaN,NaN,NaN,NaN
1,161936,2015,2,4,60,60,60,45427,28528,8.65145,...,"[Campaign Games, Legacy, Pandemic]",2,2,[Chris Quilliams],[1],NaN,NaN,NaN,NaN,NaN
2,182028,2015,2,4,240,180,240,17668,14006,8.56732,...,[Tableau Building],NaN,3,"[Jakub Politzer, Milan Vavroň]",[1],NaN,NaN,NaN,NaN,NaN
3,167791,2016,1,5,120,120,120,40824,32136,8.39275,...,"[Fryxgames' Future Timeline, Planets: Mars, So...",NaN,4,[Isaac Fryxelius],[5],NaN,NaN,NaN,NaN,NaN
4,12333,2005,2,2,180,120,180,44900,32806,8.33008,...,"[Country: Soviet Union, Country: USA, Historic...",NaN,7,"[Viktor Csete, Rodger B. MacGowan, Chechu Niet...",[1],1,NaN,NaN,NaN,NaN
5,187645,2016,2,4,240,180,240,22816,14956,8.45402,...,"[Components: Miniatures, Star Wars]",3,6,[],"[1, 3, 4]",NaN,NaN,NaN,NaN,NaN
6,169786,2016,1,5,115,90,115,44416,34220,8.28321,...,"[Alternate History, Components: Miniatures, Cr...",NaN,9,[Jakub Rozalski],NaN,NaN,NaN,NaN,NaN,NaN
7,220308,2017,1,4,150,60,150,8928,7219,8.59267,...,"[Solitaire Games, Tableau Building, Terra Myst...",NaN,5,[Dennis Lohausen],NaN,NaN,NaN,NaN,NaN,NaN
8,193738,2016,2,4,150,75,150,19872,16334,8.28944,...,"[Animals: Cattle, Stronghold Games: The Great ...",NaN,8,[Andreas Resch],[1],NaN,NaN,NaN,NaN,NaN
9,120677,2012,2,5,150,60,150,34333,30969,8.21162,...,"[Tableau Building, Terra Mystica]",NaN,10,[Dennis Lohausen],[1],NaN,NaN,NaN,NaN,NaN


In [26]:
top_games[298:301]

,Name,Rank,ID
298,Biblios,299,34219
299,Small World Underground,300,97786
300,Madeira,301,95527
